In [37]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import weaviate
from weaviate.classes.init import Auth
import weaviate.classes as wvc
from weaviate.collections.classes.filters import Filter
from tqdm import tqdm

In [31]:
df = pd.read_csv("../data/clem_vulgate.csv")
df

,book,chapter,verse,latin,latin_clean
0,1Cor,1,1,Paulus vocatus Apostolus Jesu Christi per volu...,paulus vocatus apostolus jesu christi per volu...
1,1Cor,1,2,"ecclesiae Dei, quae est Corinthi, sanctificati...",ecclesiae dei quae est corinthi sanctificatis ...
2,1Cor,1,3,"Gratia vobis, et pax a Deo Patre nostro, et Do...",gratia vobis et pax a deo patre nostro et domi...
3,1Cor,1,4,Gratias ago Deo meo semper pro vobis in gratia...,gratias ago deo meo semper pro vobis in gratia...
4,1Cor,1,5,"quod in omnibus divites facti estis in illo, i...",quod in omnibus divites facti estis in illo in...
...,...,...,...,...,...
35804,Zach,14,17,Et erit: qui non ascenderit de familiis terrae...,et erit qui non ascenderit de familiis terrae ...
35805,Zach,14,18,Quod etsi familia Aegypti non ascenderit et no...,quod etsi familia aegypti non ascenderit et no...
35806,Zach,14,19,"Hoc erit peccatum Aegypti, et hoc peccatum omn...",hoc erit peccatum aegypti et hoc peccatum omni...
35807,Zach,14,20,"In die illa, erit quod super frenum equi est, ...",in die illa erit quod super frenum equi est sa...


In [30]:
model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(df.latin.tolist())

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Applications/anaconda3/envs/tap/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

/Applications/anaconda3/envs/tap/lib/python3.10/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join

In [32]:
df["embedding"] = list(embeddings)

In [33]:
df

,book,chapter,verse,latin,latin_clean,embedding
0,1Cor,1,1,Paulus vocatus Apostolus Jesu Christi per volu...,paulus vocatus apostolus jesu christi per volu...,"[-0.055635616, -0.0065089255, -0.043669138, 0...."
1,1Cor,1,2,"ecclesiae Dei, quae est Corinthi, sanctificati...",ecclesiae dei quae est corinthi sanctificatis ...,"[-0.019228704, -0.05051961, -0.024509165, -0.0..."
2,1Cor,1,3,"Gratia vobis, et pax a Deo Patre nostro, et Do...",gratia vobis et pax a deo patre nostro et domi...,"[-0.03677308, -0.05688072, -0.00424547, -0.016..."
3,1Cor,1,4,Gratias ago Deo meo semper pro vobis in gratia...,gratias ago deo meo semper pro vobis in gratia...,"[-0.046626147, -0.06212013, -0.042238228, -0.0..."
4,1Cor,1,5,"quod in omnibus divites facti estis in illo, i...",quod in omnibus divites facti estis in illo in...,"[-0.012506651, -0.031329703, -0.020543754, -0...."
...,...,...,...,...,...,...
35804,Zach,14,17,Et erit: qui non ascenderit de familiis terrae...,et erit qui non ascenderit de familiis terrae ...,"[0.0014634092, 0.003242624, 0.016612116, 0.027..."
35805,Zach,14,18,Quod etsi familia Aegypti non ascenderit et no...,quod etsi familia aegypti non ascenderit et no...,"[-0.032344513, 0.014963197, -0.03372779, 0.035..."
35806,Zach,14,19,"Hoc erit peccatum Aegypti, et hoc peccatum omn...",hoc erit peccatum aegypti et hoc peccatum omni...,"[0.036506668, -0.009648186, -0.04503121, -0.00..."
35807,Zach,14,20,"In die illa, erit quod super frenum equi est, ...",in die illa erit quod super frenum equi est sa...,"[-0.028904956, -0.03887993, -0.014923892, -0.0..."


In [34]:
df.to_parquet("../data/clem_vulgate_vectors.parquet")

In [35]:
df

,book,chapter,verse,latin,latin_clean,embedding
0,1Cor,1,1,Paulus vocatus Apostolus Jesu Christi per volu...,paulus vocatus apostolus jesu christi per volu...,"[-0.055635616, -0.0065089255, -0.043669138, 0...."
1,1Cor,1,2,"ecclesiae Dei, quae est Corinthi, sanctificati...",ecclesiae dei quae est corinthi sanctificatis ...,"[-0.019228704, -0.05051961, -0.024509165, -0.0..."
2,1Cor,1,3,"Gratia vobis, et pax a Deo Patre nostro, et Do...",gratia vobis et pax a deo patre nostro et domi...,"[-0.03677308, -0.05688072, -0.00424547, -0.016..."
3,1Cor,1,4,Gratias ago Deo meo semper pro vobis in gratia...,gratias ago deo meo semper pro vobis in gratia...,"[-0.046626147, -0.06212013, -0.042238228, -0.0..."
4,1Cor,1,5,"quod in omnibus divites facti estis in illo, i...",quod in omnibus divites facti estis in illo in...,"[-0.012506651, -0.031329703, -0.020543754, -0...."
...,...,...,...,...,...,...
35804,Zach,14,17,Et erit: qui non ascenderit de familiis terrae...,et erit qui non ascenderit de familiis terrae ...,"[0.0014634092, 0.003242624, 0.016612116, 0.027..."
35805,Zach,14,18,Quod etsi familia Aegypti non ascenderit et no...,quod etsi familia aegypti non ascenderit et no...,"[-0.032344513, 0.014963197, -0.03372779, 0.035..."
35806,Zach,14,19,"Hoc erit peccatum Aegypti, et hoc peccatum omn...",hoc erit peccatum aegypti et hoc peccatum omni...,"[0.036506668, -0.009648186, -0.04503121, -0.00..."
35807,Zach,14,20,"In die illa, erit quod super frenum equi est, ...",in die illa erit quod super frenum equi est sa...,"[-0.028904956, -0.03887993, -0.014923892, -0.0..."


In [36]:
WEAVIATE_URL = "https://kmijkobrxw63wdclh5pgw.c0.us-east1.gcp.weaviate.cloud"
WEAVIATE_API_KEY  = "zNipmxCSgbAThAjY8zfvm4aS7MN3b1PdzPdh"
collection_name = "Vulgate"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

if client.collections.exists(collection_name):  # In case we've created this collection before
    client.collections.delete(collection_name)  # THIS WILL DELETE ALL DATA IN THE COLLECTION

vulgate = client.collections.create(
    name=collection_name,
    properties=[
        wvc.config.Property(
            name="text",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="book",
            data_type=wvc.config.DataType.TEXT
        ),
        wvc.config.Property(
            name="chapter",
            data_type=wvc.config.DataType.INT
        ),
        wvc.config.Property(
            name="verse",
            data_type=wvc.config.DataType.INT
        ),
    ]
)

In [38]:
# Prepare all the data rows first
data_rows = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Preparing data"):
    data_rows.append({
        "properties": {
            "text": row['latin'],
            "book": row['book'],
            "chapter": int(row['chapter']),
            "verse": int(row['verse'])
        },
        "vector": row['embedding']
    })

Preparing data: 100%|██████████| 35809/35809 [00:00<00:00, 60819.73it/s]


In [40]:
# Now perform the batch insertion
with vulgate.batch.dynamic() as batch:
    for data_row in tqdm(data_rows, desc="Inserting data"):
        batch.add_object(
            properties=data_row['properties'],
            vector=data_row['vector']
        )

Inserting data:  65%|██████▌   | 23319/35809 [00:30<00:24, 512.84it/s] /Applications/anaconda3/envs/tap/lib/python3.10/site-packages/weaviate/warnings.py:303: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/Applications/anaconda3/envs/tap/lib/python3.10/asyncio/selector_events.py:710: ResourceWarning: unclosed transport <_SelectorSocketTransport fd=93 read=idle write=<idle, bufsize=0>>
  _warn(f"unclosed transport {self!r}", ResourceWarning, source=self)
Inserting data: 100%|██████████| 35809/35809 [00:55<00:00, 641.35it/s] 


In [41]:
from weaviate.classes.query import MetadataQuery


In [60]:
query = "Si quis uult post me uenire, abneget semetipsum et tollat crucem suam et sequatur me"
def find_similar(query, threshold):
    query_vector = model.encode([query])[0]
    response = vulgate.query.near_vector(
        near_vector=query_vector,
        limit=10,
        return_metadata=MetadataQuery(distance=True)
    )

    for o in response.objects:
        if o.metadata.distance < threshold:
            print(o.properties["book"], o.properties["chapter"], o.properties["verse"], ": ", o.properties["text"])
            print(o.metadata.distance)

In [67]:
query = "initio Dominus fecit terras"
find_similar(query, threshold=.4)

Gn 1 1 :  In principio creavit Deus caelum et terram.
0.28606468439102173
Hbr 1 10 :  Et: Tu in principio, Domine, terram fundasti: et opera manuum tuarum sunt caeli.
0.3451509475708008


In [46]:
text = """
GESTA FRANCORUM
LIBER I

[i] Cum iam appropinquasset ille terminus quem dominus Iesus cotidie suis demonstrat fidelibus, specialiter in euangelio dicens: "Si quis uult post me uenire, abneget semetipsum et tollat crucem suam et sequatur me" facta est igitur motio ualida per uniuersas Galliarum regiones, ut si aliquis Deum studiose puroque corde et mente sequi desideraret, atque post ipsum crucem fideliter baiulare uellet, non pigritaretur Sancti Sepulchri uiam celerius arripere. Apostolicus namque Romanae sedis ultra montanas partes quantocius profectus est cum suis archiepiscopis, episcopis, abbatibus, et presbiteris, coepitque subtiliter sermocinari et predicare, dicens, ut si quis animam suam saluam facere uellet, non dubitaret humiliter uiam incipere Domini, ac si denariorum ei deesset copia, diuina ei satis daret misericordia. Ait namque domnus apostolicus 'Fratres, uos oportet multa pati pro nomine Christi, uidelicet miserias, paupertates, nuditates, persecutiones, egestates, infirmitates, fames, sites et alia huiusmodi, sicuti Dominus ait suis discipulis: "Oportet uos pati multa pro nomine meo", et: "Nolite erubescere loqui ante facies hominum; ego uero dabo uobis os et eloquium", ac deinceps: "Persequetur uos larga retributio". Cumque iam hic sermo paulatim per uniuersas regiones ac Galliarum patrias coepisset crebrescere, Franci audientes talia protinus in dextra crucem suere scapula, dicentes sese Christi unanimiter sequi uestigia, quibus de manu erant redempti tartarea. Iamiamque Galliae suis remotae sunt domibus.

[ii] Fecerunt denique Galli tres partes. Vna pars Francorum in Hungariae intrauit regionem, scilicet Petrus Heremita, et dux Godefridus, et Balduinus frater eius, et Balduinus comes de Monte.  Isti potentissimi milites et alii plures quos ignoro uenerunt per uiam quam iamdudum Karolus Magnus mirificus rex Franciae aptari fecit usque Constantinopolim.

Petrus uero supradictus primus uenit Constantinopolim in kalendis Augusti et cum eo maxima gens Alamannorum. Illic inuenit Lombardos et Longobardos et alios plures congregatos, quibus imperator iusserat dari mercatum, sicuti erat in ciuitate, dixitque illis 'Nolite transmeare Brachium, donec ueniat maxima Christianorum uirtus, quoniam uos tanti non estis, ut cum Turcis preliari ualeatis. Ipsique Christiani nequiter deducebant se, quia palatia urbis sternebant et ardebant, et auferebant plumbum quo ecclesiae erant coopertae et uendebant Grecis. Vnde imperator iratus est iussitque eos transmeare Brachium. Postquam transfretauerunt, non cessabant agere omnia mala, comburentes et deuastantes domos et ecclesias. Tandem peruenerunt Nicomediam, ubi diuisi sunt Lombardi et Longobardi, et Alamanni a Francis, quia Franci tumebant superbia. Elegerunt Lombardi et Longobardi seniorem super se, cui nomen Rainoldus, Alamanni similiter. Et intrauerunt in Romaniam et per quatuor dies ierunt ultra Nicenam urbem inueneruntque quoddam castrum cui nomen Exerogorgo, quod erat uacuum gente. Et apprehenderunt illud, in qua inuenerunt satis frumenti et uini et carnis, et omnium bonorum abundantiam. Audientes itaque Turci quod Christiani essent in castro, uenerunt obsidere illud. Ante portam castri erat puteus, et ad pedem castri fons uiuus, iuxta quem exiit Rainaldus insidiari Turcos. Venientes uero Turci in festo sancti Michahelis, inuenerunt Rainaldum et qui cum eo erant, occideruntque Turci multos ex eis. Alii fugerunt in castrum. Quod confestim Turci obsederunt, eisque aquam abstulerunt. Fueruntque nostri in tanta afflictione sitis, ut flebotomarent suos equos et asinos, quorum sanguinem bibebant. Alii mittebant zonas atque panniculos in piscinam, et inde exprimebant aquam in os suum. Alii mingebant in pugillo alterius, et bibebant. Alii fodiebant humidam terram, et supinabant se, terramque sternebant super pectora sua, pro nimia ariditate sitis. Episcopi uero et presbiteri confortabant nostros et commonebant ne deficerent. Haec tribulatio fuit per octo dies. Denique dominus Alamannorum concordatus est cum Turcis, ut traderet socios illis, et fingens se exire ad bellum, fugit ad illos et multi cum eo. Illi autem qui Deum negare noluerunt, capitalem sententiam susceperunt. Alios quos ceperunt uiuos adinuicem diuiserunt quasi oues. Alios miserunt ad signum et sagittabant eos; alios uendebant et donabant quasi animalia. Quidam conducebant suos in domum suam, alios in Corosanum, alios in Antiochiam, alios in Aleph, aut ubi ipsi manebant. Isti primo felix acceperunt martirium pro nomine Domini Iesu.

Audientes denique Turci quod Petrus Heremita et Guualterius Sinehabere fuissent in Cyuito, quae supra Nicenam urbem est, uenerunt illuc cum magno gaudio ut occiderent illos et eos qui cum ipsis erant. Cumque uenissent obuiauerunt Guualterio cum suis, quos Turci mox occiderunt. Petrus uero Heremita paulo ante ierat Constantinopolim, eo quod nequibat refrenare illam diuersam gentem, quae nec illum nec uerba eius audire uolebat. Irruentes uero Turci super eos occiderunt multos ex eis; alios inuenerunt dormientes, alios nudos, quos onmes necauerunt, cum quibus quemdam sacerdotem inuenerunt missam celebrantem, quem statim super altare martirizauerunt. Illi uero qui euadere potuerunt Cyuito fugerunt; alii precipitabant se in mare, alii latebant in siluis et montanis. Turci uero persequentes illos in castrum adunauerunt ligna, ut eos comburerent cum castro. Christiani igitur qui in castro erant miserunt ignem in ligna congregata, et uersus ignis in Turcos quosdam eorum concremauit, sed ab illo incendio Deus nostros tunc liberauit. Tandem Turci apprehenderunt illos uiuos, diuiseruntque illos sicut prius fecerant alios, et disperserunt illos per uniuersas regiones has, alios in Corosanum, alios in Persidem. Hoc totum est factum in mense Octobri. Audiens imperator quod Turci sic dissipassent nostros, gauisus est ualde, et mandauit fecitque eos Brachium transmeare. Postquam ultra fuerunt, comparauit omnia arma eorum.

[iii] Secunda uero pars intrauit in Sclauiniae partes, scilicet comes de Sancto Egidio Raimundus et Podiensis episcopus. Tertia autem pars per antiquam Romae uiam uenit. In hac parte fuerunt Boamundus, et Richardus de Principatu, Rotbertus comes Flandrensis, Rotbertus Nortmannus, Hugo Magnus, Eurardus de Puisatio, Achardus de Monte Merloi, Isuardus de Musone, et alii plures. Deinde uenerunt ad portum Brandosim aut Barim siue Otrentum. Hugo denique Magnus et Willelmus Marchisi filius intrauerunt mare ad portum Bari, et transfretantes uenerunt Durachium. Audiens uero dux illius loci hos prudentissimos uiros illic esse applicatos, mox mala cogitatio cor eius tetigit, illosque apprehendit, ac iussit Constantinopolim imperatori caute duci, quo ei fidelitatem facerent.

Dux denique Godefridus primus omnium seniorum Constantinopolim uenit cum magno exercitu, duobus diebus ante Domini nostri Natale, et hospitatus est extra urbem, donec iniquus imperator iussit eum hospitari in burgo urbis. Cumque fuisset hospitatus dux, secure mittebat armigeros suos per singulos dies, ut paleas et alia equis necessaria asportarent. Et cum putarent exire fiducialiter quo uellent, iniquus imperator Alexius imperauit Turcopolis et Pinzinacis inuadere illos et occidere. Balduinus itaque frater ducis haec audiens misit se in insidiis, tandemque inuenit eos occidentes gentem suam, eosque inuasit forti animo, ac Deo iuuante superauit eos. Et apprehendens sexaginta ex eis, partem occidit, partem duci fratri suo presentauit. Quod cum audisset imperator, ualde iratus est. Videns uero dux inde iratum imperatorem, exiit cum suis de burgo et hospitatus est extra urbem. Sero autem facto, infelix imperator iussit suis exercitibus inuadere ducem cum Christi gente. Quos dux persequens inuictus cum Christi militibus septem ex illis occidit, persequendo alios usque ad portam ciuitatis. Reuersusque dux ad sua tentoria mansit inibi per quinque dies, donec pactum iniit cum imperatore, dixitque illi imperator ut transfretaret Brachium sancti Georgii, permisitque eum habere omnem mercatum ibi sicut est Constantinopoli; et pauperibus elemosinam erogare, unde potuissent uiuere.

[iiii] At bellipotens Boamundus qui erat in obsidione Malfi, Scafardi Pontis, audiens uenisse innumerabilem gentem Christianorum de Francis, ituram ad Domini Sepulchrum, et paratam ad prelium contra gentem paganorum, coepit diligenter inquirere quae arma pugnandi haec gens deferat, et quam ostensionem Christi in uia portet, uel quod signum in certamine sonet. Cui per ordinem haec dicta sunt: 'Deferunt arma ad bellum congrua, in dextra uel inter utrasque scapulas crucem Christi baiulant; sonum uero "Deus uult, Deus uult, Deus uult!" una uoce conclamant. Mox Sancto commotus Spiritu, iussit preciosissimum pallium quod apud se habebat incidi, totumque statim in cruces expendit. Coepit tunc ad eum uehementer concurrere maxima pars militum qui erant in obsidione illa, adeo ut Rogerius comes pene solus remanserit, reuersusque Siciliam dolebat et merebat quandoque gentem amittere suam. Denique reuersus iterum in terram suam dominus Boamundus diligenter honestauit sese ad incipiendum Sancti Sepulchri iter. Tandem transfretauit mare cum suo exercitu, et cum eo Tancredus Marchisi filius, et Richardus princeps, ac Rainulfus frater eius, et Rotbertus de Ansa, et Hermannus de Canni, et Rotbertus de Surda Valle, et Robertus filius Tostani, et Hunfredus filius Radulfi, et Ricardus filius comitis Rainulfi, et comes de Russignolo cum fratribus suis, et Boello Carnotensis, et Alberedus de Cagnano, et Hunfredus de Monte Scabioso. Hi omnes transfretauerunt ad Boamundi famulatum, et applicuerunt Bulgariae partibus; ubi inuenerunt nimiam abundantiam frumenti et uini et alimentorum corporis. Deinde descendentes in uallem de Andronopoli; expectauerunt gentem suam, donec omnes pariter transfretassent. Tunc Boamundus ordinauit concilium cum gente sua, confortans et monens omnes ut boni et humiles essent; et ne depredarentur terram istam quia Christianorum erat, et nemo acciperet nisi quod ei sufficeret ad edendum.

Tunc exeuntes inde, uenerunt per nimiam plenitudinem de uilla in uillam, de ciuitate in ciuitatem, de castello in castellum, quousque peruenimus Castoriam; ibique Natiuitatem Domini solemniter celebrauimus; fuimusque ibi per plures dies, et quesiuimus mercatum, sed ipsi noluerunt nobis assentire, eo quod ualde timebant nos, non putantes nos esse peregrinos, sed uelle populari terram et occidere illos. Quapropter apprehendebamus boues, equos et asinos, et omnia quae inueniebamus. Egressi de Castoria, intrauimus Palagoniam, in qua erat quoddam hereticorum castrum. Quod undique aggressi sumus, moxque nostro succubuit imperio. Accenso itaque igne, combussimus castrum cum habitatoribus suis. Postea peruenimus ad flumen Bardarum. Denique perrexit dominus Boamundus ultracum sua gente, sed non tota. Remansit enim ibi comes de Russignolo, cum fratribus suis. Venit exercitus imperatoris, et inuasit comitem cum fratribus suis, et omnes qui erant cum eis. Quod audiens Tancredus rediit retro, et proiectus in flumen natando peruenit ad alios, et duo milia miserunt se in flumen sequendo Tancredum. Tandem inuenerunt Turcopulos et Pinzinacos dimicantes cum nostris. Quos repente fortiter inuaserunt, et prudenter eos superauerunt. Et apprehenderunt plures ex illis, et duxerunt illos ligatos ante domini Boamundi presentiam. Quibus ait ipse, 'Quare miseri occiditis gentem Christi et meam? Ego cum uestro imperatore nullam altercationem habeo. Qui responderunt: 'Nos nequimus aliud agere. In roga imperatoris locati sumus, et quicquid nobis imperat nos oportet implere. Quos Boamundus impunitos permisit abire. Hoc bellum factum est in quarta feria, quae est caput ieiunii. Per omnia benedictus Deus. Amen.
"""

In [47]:
import spacy

In [48]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

/Applications/anaconda3/envs/tap/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [49]:
sentences = [sent.text for sent in nlp(text).sents]

In [50]:
len(sentences)

93

In [54]:
for sent in sentences:
    query_vector = model.encode([sent])[0]
    response = vulgate.query.near_vector(
        near_vector=query_vector,
        limit=10,
        return_metadata=MetadataQuery(distance=True)
    )

    for o in response.objects:
        if o.metadata.distance < .4:
            print(F"MATCH: {sent}")
            print(o.metadata.distance)
            print(o.properties)

MATCH: 
GESTA FRANCORUM
LIBER I

[i] Cum iam appropinquasset ille terminus quem dominus Iesus cotidie suis demonstrat fidelibus, specialiter in euangelio dicens: "Si quis uult post me uenire, abneget semetipsum et tollat crucem suam et sequatur me" facta est igitur motio ualida per uniuersas Galliarum regiones, ut si aliquis Deum studiose puroque corde et mente sequi desideraret, atque post ipsum crucem fideliter baiulare uellet, non pigritaretur Sancti Sepulchri uiam celerius arripere.
0.39440619945526123
{'chapter': 16, 'text': 'Tunc Jesus dixit discipulis suis: Si quis vult post me venire, abneget semetipsum, et tollat crucem suam, et sequatur me.', 'book': 'Mt', 'verse': 24}
MATCH: Et apprehenderunt illud, in qua inuenerunt satis frumenti et uini et carnis, et omnium bonorum abundantiam.
0.35843056440353394
{'text': 'Congere frusta ejus in eam, omnem partem bonam, femur et armum, electa et ossibus plena.', 'chapter': 24, 'book': 'Ez', 'verse': 4}
MATCH: Fueruntque nostri in tanta a